In [1]:
# https://sparkbyexamples.com/spark/spark-read-binary-file-into-dataframe/
df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/{00*,01*}")
df.printSchema()
df.count()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1650412208045_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

27288

## Use the built-in ```toNDArray``` function

In [2]:
# https://stackoverflow.com/a/69215982/11262633
import pyspark.sql.functions as F
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT

@F.udf(returnType=VectorUDT())
def img2vec(x):
    try:
        image_np = DenseVector(ImageSchema.toNDArray(x).flatten())
    except:
        image_np = None
    return image_np

print(f'Image fields = {ImageSchema.imageFields}')
df_new = df.withColumn('vecs',img2vec('image'))
df_new.show()
#df_new.select('vecs').first().asDict().keys()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Image fields = ['origin', 'height', 'width', 'nChannels', 'mode', 'data']
+--------------------+--------------------+
|               image|                vecs|
+--------------------+--------------------+
|{s3://multimedia-...|[197.0,198.0,198....|
|{s3://multimedia-...|                null|
|{s3://multimedia-...|[0.0,149.0,128.0,...|
|{s3://multimedia-...|[40.0,52.0,153.0,...|
|{s3://multimedia-...|[127.0,111.0,229....|
|{s3://multimedia-...|[42.0,143.0,128.0...|
|{s3://multimedia-...|[253.0,253.0,253....|
|{s3://multimedia-...|[14.0,10.0,9.0,10...|
|{s3://multimedia-...|[45.0,108.0,52.0,...|
|{s3://multimedia-...|[104.0,53.0,33.0,...|
|{s3://multimedia-...|[0.0,0.0,0.0,0.0,...|
|{s3://multimedia-...|[209.0,197.0,101....|
|{s3://multimedia-...|[118.0,100.0,71.0...|
|{s3://multimedia-...|[244.0,248.0,253....|
|{s3://multimedia-...|[238.0,250.0,254....|
|{s3://multimedia-...|[131.0,220.0,217....|
|{s3://multimedia-...|[219.0,194.0,174....|
|{s3://multimedia-...|[1.0,1.0,1.0,0.0,...|
|{

In [3]:
df_new.where(df_new.vecs.isNull()).select('image.origin').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/4/statements/3 with error payload: {"msg":"requirement failed: Session isn't active."}


In [5]:
df_new.count()

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/2 with error payload: {"msg":"Session '2' not found."}
